In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input
from tensorflow.keras.models import load_model, Model

In [2]:
# Load data from either .csv or .feather
currency_data = pd.read_feather("data/banknote_net.feather") # feather is faster and more robust than csv.
currency_data.head()

,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,...,v_248,v_249,v_250,v_251,v_252,v_253,v_254,v_255,Currency,Denomination
0,0.423395,0.327657,2.568988,3.166228,4.801421,5.531792,2.458083,1.218453,0.000000,1.116785,...,0.000000,2.273451,5.790633,0.000000,0.000000,0.0,5.635400,0.000000,AUD,100_1
1,1.158823,1.669602,3.638447,2.823524,4.839890,2.777757,0.753350,0.764005,0.347871,1.928572,...,0.000000,2.329623,3.516146,0.000000,0.000000,0.0,2.548191,1.053410,AUD,100_1
2,0.000000,0.958235,4.706119,1.688242,3.312702,4.516483,0.000000,1.876461,2.250795,1.883192,...,0.811282,5.591417,1.879267,0.641139,0.571079,0.0,1.861483,2.172145,AUD,100_1
3,0.920511,1.820294,3.939334,3.206829,6.253655,0.942557,2.952453,0.000000,2.064298,1.367196,...,1.764936,3.415151,2.518404,0.582229,1.105192,0.0,1.566918,0.533945,AUD,100_1
4,0.331918,0.000000,3.330771,3.023437,4.369099,5.177336,1.499362,0.590646,0.553625,1.405708,...,0.000000,4.615945,4.825463,0.302261,0.378229,0.0,2.710654,0.325945,AUD,100_1


In [3]:
# Filter out Euro notes
filter=currency_data.Currency=="EUR"
currency_data=currency_data[filter]

In [4]:
labels = pd.get_dummies(currency_data['Denomination'].astype("category"))

In [5]:
# Load the trained classifier model from save file
final_model = load_model("models/best_model.h5")

<img src='test.jpg'></img>

In [6]:
# Define image size
IMG_SIZE = (224,224)

# Load the pre-trained encoder model
encoder_model = load_model("models/banknote_net_encoder.h5")

# Input tensor with shape (IMG_SIZE[0], IMG_SIZE[1], 3)
input1 = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# Apply the encoder model to transform input1 to the desired shape (256,)
output_of_encoder = encoder_model(input1)

# Create a new model with the transformed input
model_with_transformed_input = Model(inputs=input1, outputs=output_of_encoder)

# Function to predict class from an image
def predict_image_class(image_path, encoder_model, classifier_model):
    # Load and preprocess the image
    input_image = tf.keras.preprocessing.image.load_img(image_path, target_size=IMG_SIZE)
    input_image_array = tf.keras.preprocessing.image.img_to_array(input_image)
    input_image_array = tf.expand_dims(input_image_array, axis=0)  # Add batch dimension

    # Transform the input using the encoder model
    transformed_input = model_with_transformed_input.predict(input_image_array)
    # Use the pre-trained classifier model to predict the class
    predictions = classifier_model.predict(transformed_input)
    predicted_class = np.argmax(predictions, axis=1)

    return predicted_class[0], predictions

# Example usage:
image_path = "test.jpg"
predicted_class, prediction_probabilities = predict_image_class(image_path, model_with_transformed_input, final_model)

# Get the class label from one-hot encoded labels
predicted_label = list(labels.columns)[predicted_class]
print(f"Predicted label: {predicted_label}")
print(f"Prediction probabilities: {prediction_probabilities}")

1/1 [==============================] - 0s 73ms/step
Predicted label: 50_1
Prediction probabilities: [[0.02690291 0.21812914 0.00561177 0.03726324 0.01002101 0.09501909
  0.01177684 0.00190005 0.5480136  0.00338407 0.03688015 0.00509808]]
